In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

In [4]:
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
langsmith_key = os.getenv("LANGSMITH_API_KEY")

os.environ["LANGCHAIN_API_KEY"] = langsmith_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "projecr-agent"

In [5]:
llm = AzureChatOpenAI(
        model=os.getenv("OPENAI_CHAT_MODEL"),
        deployment_name=os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        azure_endpoint=os.getenv("API_BASE"),
        openai_api_version=os.getenv("API_VERSION")
    )
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000013F348A0CE0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000013F348A2C30> root_client=<openai.lib.azure.AzureOpenAI object at 0x0000013F346244A0> root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000013F348A0E30> model_name='gpt-35-turbo' model_kwargs={} openai_api_key=SecretStr('**********') disabled_params={'parallel_tool_calls': None} azure_endpoint='https://mutualizedopenai.openai.azure.com' deployment_name='text-embedding-ada-002' openai_api_version='2023-05-15' openai_api_type='azure'


start building chatbot using langgraph

In [6]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [7]:
class State(TypedDict):
    messages : Annotated[list, add_messages]
    
graph_builder= StateGraph(State)

In [8]:
graph_builder 

In [9]:
def chatbot(state: State):
    return {"message": llm.invoke(state["messages"])}

In [10]:
graph_builder.add_node("chatbot", chatbot)

In [11]:
graph_builder

In [12]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [13]:
graph = graph_builder.compile()

In [14]:
from IPython.display import display, Image
try:
    display(Image(graph.get_graph().draw_mermaind_png()))
except Exception:
    pass

In [15]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["q", "quit"]:
        print("Goodbye!")
        break
    for event in graph.stream({'messages':("user",user_input)}):
        print(event.values())
        for value in event.values():
            print(value["message"])
            print("Assistant:", value["message"].content)

BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The chatCompletion operation does not work with the specified model, text-embedding-ada-002. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}